In [1]:
import numpy as np
import pandas as pd
import requests
import config
import os
import json
from pandas.io.json import json_normalize
import pygsheets

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_

In [2]:
### Maine Committee Search ###
#Set search for all Maine committees
state = 'ME'
office = 'P'
cycle='2020'


#Initialize dataframe collector for itemized contribs
dfs=[]
id=int(0)
p=int(1)

querydict = {'api_key':config.api_key
            ,'state':state
            ,'office':office
            ,'cycle':cycle
            ,'per_page':'100'
            ,'page':p}
    
r = requests.get('https://api.open.fec.gov/v1/committees/',params=querydict).json()
    
#Loop through pages
while p <= r['pagination']['pages']:

    print('page: '+str(p)+','+str(r['pagination']['per_page'])+' records')

    querydict.update(page=p)
    #Pull results
    r = requests.get('https://api.open.fec.gov/v1/committees/',params=querydict).json()

    #Add results to dataframe
    df = json_normalize(r['results'])
    dfs.append(df)
    p=p+1

#Reset to page 1
p=int(1)
#Increment to next candidate
id=id+1

print('Job Complete')

page: 1,100 records
Job Complete


In [3]:
pres = df[df['committee_type_full']=='Presidential']
pres

,affiliated_committee_name,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,name,organization_type,organization_type_full,party,party_full,state,treasurer_name
7,NONE,[P00007856],C00681205,P,Presidential,"[2018, 2020]",P,Principal campaign committee,Q,2018-06-18,2018-06-18,2018-06-18,"BRUCE JOHN KENNEWAY, INDEPENDENT PRESIDENT, NO...",None,None,W,WRITE-IN,ME,"KENNEWAY, BRUCE JOHN MR"
9,NONE,[P00008169],C00685867,P,Presidential,"[2018, 2020]",P,Principal campaign committee,Q,2018-08-22,2018-08-22,2018-08-22,CALEB GAUL FOR PRESIDENT,None,None,DEM,DEMOCRATIC PARTY,ME,"GAUL, CALEB"
10,NONE,[P00013169],C00715995,P,Presidential,[2020],P,Principal campaign committee,Q,2019-08-16,2019-08-16,2019-10-15,CAMPAIGN COMMITTEE TO ELECT EVRET J GREER,None,None,LIB,LIBERTARIAN PARTY,ME,"GREER, EVRET JUSTIN"
11,None,[P00012708],C00711606,P,Presidential,[2020],P,Principal campaign committee,T,2019-07-09,2019-07-09,2019-10-11,CITIZENS FOR TOM KAWCZYNSKI,None,None,REP,REPUBLICAN PARTY,ME,THOMAS J KAWCZYNSKI
23,NONE,[P00005942],C00641100,P,Presidential,"[2018, 2020]",P,Principal campaign committee,Q,2017-05-22,2017-05-22,2019-10-16,FRIENDS OF FRED WIAND,None,None,DEM,DEMOCRATIC PARTY,ME,"WIAND, FRED"


In [25]:
#Write itemized individual results to local CSV
cwd = os.getcwd()
itemdf.to_csv(cwd+'/data/maine-presidential-receipts.csv')